# Real experiment evaluation

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

np.set_printoptions(precision=2)

In [ ]:
datafile = 'experiments/robot_test/circle.csv'
#datafile = 'experiments/robot_test/full_bandlimited1.csv'

anchorsfile = 'experiments/robot_test.csv'

tango_system_id = 7585
rtt_system_id = 7592

rtt_counter = -1
tango_counter = -1

def add_name(row):
    global rtt_counter
    global tango_counter 
    if row.system_id == tango_system_id:
        tango_counter += 1
        return 'Tango {}'.format(tango_counter)
    elif row.system_id == rtt_system_id:
        rtt_counter += 1
        return 'RTT {}'.format(rtt_counter)
    
    
def clean_anchor_id(row):
    return row.anchor_id.strip()


data_df = pd.read_csv(datafile)
anchors_df = pd.read_csv(anchorsfile)
anchors_df.loc[:, 'anchor_id'] = anchors_df.apply(lambda row: clean_anchor_id(row),  axis=1)
anchors_df.loc[:, 'anchor_name'] = anchors_df.apply(lambda row: add_name(row), axis=1) 
anchors_df

In [ ]:
import seaborn as sns

data_df.loc[:, 'seconds'] = (data_df.timestamp.values - data_df.timestamp.min()) / 1000. # in seconds

plt.figure()
sns.scatterplot(data=data_df[data_df.system_id==tango_system_id], x='px', y='py', hue='seconds', linewidth=0.0)
sns.scatterplot(data=anchors_df, x='px', y='py', hue='anchor_name', linewidth=0.0, style='system_id', legend=False)
plt.arrow(0, 0, 1, 0, color='red', head_width=0.2)
plt.arrow(0, 0, 0, 1, color='green', head_width=0.2)
delta = 0.1
for i, a in anchors_df.iterrows():
    plt.annotate(a.anchor_name, (a.px+delta, a.py+delta))
plt.legend(loc='lower right', bbox_to_anchor=[1.0, 0.0])
plt.axis('equal')
plt.title(datafile)

In [ ]:
plt.figure()
data = data_df[data_df.system_id==tango_system_id]
sns.scatterplot(data=data, x='seconds', y='px', linewidth=0.0, label='x')
sns.scatterplot(data=data, x='seconds', y='py', linewidth=0.0, label='y')
sns.scatterplot(data=data, x='seconds', y='pz', linewidth=0.0, label='z')
plt.legend()

In [ ]:
current_gt = None
for i, row in data_df.iterrows():
    if row.system_id == tango_system_id:
        current_gt = [row.px, row.py, row.pz]
    elif current_gt is not None:
        data_df.loc[i, ['px', 'py', 'pz']] = current_gt
    else:
        print(i, 'no gt yet.')

In [ ]:
def distance_error(row):
    if row.system_id == tango_system_id:
        return 0.0
    anchor_coord = anchors_df.loc[anchors_df.anchor_id==row.anchor_id, ['px', 'py', 'pz']].values.astype(np.float32)
    point_coord = np.array([row.px, row.py, row.pz], dtype=np.float32)
    return np.linalg.norm(point_coord - anchor_coord)
        
data_df.loc[:, 'distance_gt'] = data_df.apply(lambda row: distance_error(row), axis=1)
data_df.loc[:, 'distance_error'] = np.abs(data_df.distance_gt - data_df.distance)

In [ ]:
import itertools
colors = itertools.cycle(plt.get_cmap('tab10').colors)
rtt_ids = anchors_df[anchors_df.system_id==rtt_system_id].anchor_id.unique()
fig, axs = plt.subplots(1, len(rtt_ids), sharey=True)
fig.set_size_inches(10, 5)
for ax, anchor_id in zip(axs, rtt_ids):
    color = next(colors)
    data = data_df[data_df.anchor_id==anchor_id]
    anchor_name = anchors_df.loc[anchors_df.anchor_id==anchor_id, 'anchor_name']
    ax.plot(data.seconds, data.distance_gt, linestyle=':', label=anchor_name, color=color)
    ax.plot(data.seconds, data.distance, linestyle='-', color=color)
    ax.set_ylim(0, 15)
#sns.scatterplot(data=data_df[data_df.system_id==rtt_system_id], x='seconds', y='distance_error',hue='anchor_id')